In [1]:
import numpy as np
#standard Qiskit libraries
import qiskit
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

In [2]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.0', 'qiskit-aer': '0.11.0', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.0', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [3]:
molecule = "Be .0 .0 .0; H .0 .0 -1.7; H .0 .0 1.7"
driver = PySCFDriver(atom=molecule)

In [4]:
qmolecule = driver.run()

In [5]:
fct = [FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3,4])]#[FreezeCoreTransformer(freeze_core=False)] 

In [6]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver, fct)
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

# Mapping Fermions to Qubits
#num_particles = (problem.molecule_data_transformed.num_alpha,
             #problem.molecule_data_transformed.num_beta)
num_particles = problem.num_particles
qubit_op = converter.convert(main_op, num_particles = problem.num_particles)
from qiskit_nature.circuit.library import HartreeFock


num_spin_orbitals = problem.num_spin_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

In [7]:
help(init_state)

Help on HartreeFock in module qiskit_nature.circuit.library.initial_states.hartree_fock object:

class HartreeFock(qiskit.circuit.quantumcircuit.QuantumCircuit)
 |  HartreeFock(num_spin_orbitals: int, num_particles: Tuple[int, int], qubit_converter: qiskit_nature.converters.second_quantization.qubit_converter.QubitConverter) -> None
 |  
 |  A Hartree-Fock initial state.
 |  
 |  Method resolution order:
 |      HartreeFock
 |      qiskit.circuit.quantumcircuit.QuantumCircuit
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, num_spin_orbitals: int, num_particles: Tuple[int, int], qubit_converter: qiskit_nature.converters.second_quantization.qubit_converter.QubitConverter) -> None
 |      Args:
 |          num_spin_orbitals: The number of spin orbitals, has a min. value of 1.
 |          num_particles: The number of particles as a tuple storing the number of alpha- and
 |                         beta-spin electrons in the first and second number, respectivel

In [8]:
# second_q_ops['ParticleNumber']

In [27]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Parameters for q-UCC antatze
num_particles = problem.num_particles
num_spin_orbitals = problem.num_spin_orbitals
n = qubit_op.num_qubits
qc = QuantumCircuit(qubit_op.num_qubits)
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
#the variational parameter
p=1
qubit_label = 0
# qc.ry(theta, range(n))
#qc.rz(theta, range(n))
for k in range(1):
    for i in range(n):
        theta = Parameter(f"ry_theta{p}" )
        qc.ry(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
    for i in range(n-1):
        qc.cz(i, i+1)
    for i in range(n):
        theta = Parameter(f"ry_theta{p}" )
        qc.ry(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
#qc.rz(theta, range(n))

# Add the initial state
ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

In [28]:
qc.draw()

┌───┐      ┌───────────────┐   ┌───────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(ry_theta1) ├─■─┤ Ry(ry_theta7) ├─────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │ └───────────────┘┌───────────────┐»
q_1: ┤ Ry(ry_theta2) ├──────────────────■─────────■────────┤ Ry(ry_theta8) ├»
     ├───────────────┤                            │        └───────────────┘»
q_2: ┤ Ry(ry_theta3) ├────────────────────────────■────────────────■────────»
     └─────┬───┬─────┘┌───────────────┐                            │        »
q_3: ──────┤ X ├──────┤ Ry(ry_theta4) ├────────────────────────────■────────»
     ┌─────┴───┴─────┐└───────────────┘                                     »
q_4: ┤ Ry(ry_theta5) ├──────────────────────────────────────────────────────»
     ├───────────────┤                                                      »
q_5: ┤ Ry(ry_theta6) ├──────────────────────────────────────────────────────»
     └───────────────┘                                                      »
«                                                          
«q_0: ─────────────────────────────────────────────────────
«                                                          
«q_1: ─────────────────────────────────────────────────────
«     ┌───────────────┐                                    
«q_2: ┤ Ry(ry_theta9) ├────────────────────────────────────
«     └───────────────┘┌────────────────┐                  
«q_3: ────────■────────┤ Ry(ry_theta10) ├──────────────────
«             │        └────────────────┘┌────────────────┐
«q_4: ────────■────────────────■─────────┤ Ry(ry_theta11) ├
«                              │         ├────────────────┤
«q_5: ─────────────────────────■─────────┤ Ry(ry_theta12) ├
«                                        └────────────────┘

In [29]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -3.4845488376895197
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -18.169420766835
  - computed part:      -3.48454883769
  - FreezeCoreTransformer extracted energy part: -14.684871929146
~ Nuclear repulsion energy (Hartree): 2.6458860546
> Total ground state energy (Hartree): -15.523534712235
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [30]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
optimizer = COBYLA(maxiter=15000)
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
def callback(eval_count, parameters, mean, std):  
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 131,
    'eigenstate': array([-5.32770612e-10+0.j, -5.09532682e-05+0.j, -1.14024939e-09+0.j,
       -2.35160046e-09+0.j,  2.02781568e-11+0.j,  1.90746172e-06+0.j,
       -4.45011862e-11+0.j,  1.48162014e-08+0.j,  1.04560580e-05+0.j,
        9.99999993e-01+0.j,  2.23797805e-05+0.j,  4.59081707e-05+0.j,
       -5.72541723e-11+0.j,  8.49822148e-06+0.j,  3.83750564e-08+0.j,
       -6.45003375e-06+0.j,  7.95874706e-11+0.j,  7.61161349e-06+0.j,
        1.70346735e-10+0.j,  3.49351653e-10+0.j,  1.35939918e-13+0.j,
        1.28976042e-08+0.j,  5.92335595e-15+0.j,  4.83512863e-11+0.j,
       -9.29788103e-10+0.j, -8.89233876e-05+0.j, -1.99008591e-09+0.j,
       -4.08231019e-09+0.j,  5.25251107e-15+0.j, -7.40514611e-10+0.j,
       -3.41277846e-12+0.j,  5.73674094e-10+0.j, -1.21436750e-14+0.j,
       -1.16140025e-09+0.j, -2.59901625e-14+0.j, -5.36083727e-14+0.j,
        4.74132654e-16+0.j,  4.45996317e-11+0.j, -1.03935599e-15+0.j,
  

In [31]:
ansatz.draw()

┌───┐      ┌───────────────┐   ┌───────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(ry_theta1) ├─■─┤ Ry(ry_theta7) ├─────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │ └───────────────┘┌───────────────┐»
q_1: ┤ Ry(ry_theta2) ├──────────────────■─────────■────────┤ Ry(ry_theta8) ├»
     ├───────────────┤                            │        └───────────────┘»
q_2: ┤ Ry(ry_theta3) ├────────────────────────────■────────────────■────────»
     └─────┬───┬─────┘┌───────────────┐                            │        »
q_3: ──────┤ X ├──────┤ Ry(ry_theta4) ├────────────────────────────■────────»
     ┌─────┴───┴─────┐└───────────────┘                                     »
q_4: ┤ Ry(ry_theta5) ├──────────────────────────────────────────────────────»
     ├───────────────┤                                                      »
q_5: ┤ Ry(ry_theta6) ├──────────────────────────────────────────────────────»
     └───────────────┘                                                      »
«                                                          
«q_0: ─────────────────────────────────────────────────────
«                                                          
«q_1: ─────────────────────────────────────────────────────
«     ┌───────────────┐                                    
«q_2: ┤ Ry(ry_theta9) ├────────────────────────────────────
«     └───────────────┘┌────────────────┐                  
«q_3: ────────■────────┤ Ry(ry_theta10) ├──────────────────
«             │        └────────────────┘┌────────────────┐
«q_4: ────────■────────────────■─────────┤ Ry(ry_theta11) ├
«                              │         ├────────────────┤
«q_5: ─────────────────────────■─────────┤ Ry(ry_theta12) ├
«                                        └────────────────┘

In [32]:
trans_ansatz = transpile(ansatz, optimization_level = 3)

In [33]:
parameter = result.optimal_parameters

In [34]:
parameter

{Parameter(ry_theta1): -0.005930304442720739,
 Parameter(ry_theta10): 0.002334308293627237,
 Parameter(ry_theta11): 0.006637195221102754,
 Parameter(ry_theta12): -0.0010884351488671474,
 Parameter(ry_theta2): 0.007547564375080672,
 Parameter(ry_theta3): 0.0017093244427710355,
 Parameter(ry_theta4): -0.002232455540095935,
 Parameter(ry_theta5): 0.0068150639644816305,
 Parameter(ry_theta6): 0.0011351725385682152,
 Parameter(ry_theta7): 0.0059092213650241,
 Parameter(ry_theta8): 0.007639237337124865,
 Parameter(ry_theta9): 0.0017262671555453516}

In [35]:
op_list = qubit_op.primitive.group_commuting()

In [36]:
help(qubit_op.primitive)

Help on SparsePauliOp in module qiskit.quantum_info.operators.symplectic.sparse_pauli_op object:

class SparsePauliOp(qiskit.quantum_info.operators.linear_op.LinearOp)
 |  SparsePauliOp(data, coeffs=None, *, ignore_pauli_phase=False, copy=True)
 |  
 |  Sparse N-qubit operator in a Pauli basis representation.
 |  
 |  This is a sparse representation of an N-qubit matrix
 |  :class:`~qiskit.quantum_info.Operator` in terms of N-qubit
 |  :class:`~qiskit.quantum_info.PauliList` and complex coefficients.
 |  
 |  It can be used for performing operator arithmetic for hundred of qubits
 |  if the number of non-zero Pauli basis terms is sufficiently small.
 |  
 |  The Pauli basis components are stored as a
 |  :class:`~qiskit.quantum_info.PauliList` object and can be accessed
 |  using the :attr:`~SparsePauliOp.paulis` attribute. The coefficients
 |  are stored as a complex Numpy array vector and can be accessed using
 |  the :attr:`~SparsePauliOp.coeffs` attribute.
 |  
 |  .. rubric:: Data

In [37]:
len(qubit_op.primitive.paulis)

165

In [38]:
paulis = qubit_op.primitive.paulis

In [39]:
qubit_op.primitive.paulis[0].to_label()

'IIIYIY'

In [40]:
help(qubit_op.primitive.paulis[0])

Help on Pauli in module qiskit.quantum_info.operators.symplectic.pauli object:

class Pauli(qiskit.quantum_info.operators.symplectic.base_pauli.BasePauli)
 |  Pauli(data=None, x=None, *, z=None, label=None)
 |  
 |  N-qubit Pauli operator.
 |  
 |  This class represents an operator :math:`P` from the full :math:`n`-qubit
 |  *Pauli* group
 |  
 |  .. math::
 |  
 |      P = (-i)^{q} P_{n-1} \otimes ... \otimes P_{0}
 |  
 |  where :math:`q\in \mathbb{Z}_4` and :math:`P_i \in \{I, X, Y, Z\}`
 |  are single-qubit Pauli matrices:
 |  
 |  .. math::
 |  
 |      I = \begin{pmatrix} 1 & 0  \\ 0 & 1  \end{pmatrix},
 |      X = \begin{pmatrix} 0 & 1  \\ 1 & 0  \end{pmatrix},
 |      Y = \begin{pmatrix} 0 & -i \\ i & 0  \end{pmatrix},
 |      Z = \begin{pmatrix} 1 & 0  \\ 0 & -1 \end{pmatrix}.
 |  
 |  **Initialization**
 |  
 |  A Pauli object can be initialized in several ways:
 |  
 |      ``Pauli(obj)``
 |          where ``obj`` is a Pauli string, ``Pauli`` or
 |          :class:`~qiskit.q

In [41]:
qubit_op.primitive.coeffs

array([ 9.11014044e-03+0.j,  3.67647998e-02+0.j, -2.76546594e-02+0.j,
        3.86820581e-02+0.j, -3.86820581e-02+0.j, -3.86820581e-02+0.j,
        3.86820581e-02+0.j, -9.20773672e-03+0.j, -9.20773672e-03+0.j,
        9.20773672e-03+0.j,  9.20773672e-03+0.j, -2.30257569e-02+0.j,
        2.30257569e-02+0.j,  2.30257569e-02+0.j, -2.30257569e-02+0.j,
        3.43707071e-02+0.j, -3.43707071e-02+0.j, -3.43707071e-02+0.j,
        3.43707071e-02+0.j,  2.16293871e-02+0.j,  2.16293871e-02+0.j,
        2.16293871e-02+0.j,  2.16293871e-02+0.j,  2.05433774e-02+0.j,
        2.05433774e-02+0.j,  2.05433774e-02+0.j,  2.05433774e-02+0.j,
        1.96289171e-03+0.j,  1.96289171e-03+0.j, -2.83235529e-03+0.j,
       -2.83235529e-03+0.j, -6.42009268e-03+0.j, -6.42009268e-03+0.j,
       -9.91087935e-03+0.j, -9.91087935e-03+0.j, -2.51940243e-03+0.j,
       -2.51940243e-03+0.j, -5.14074256e-03+0.j, -5.14074256e-03+0.j,
       -9.20773672e-03+0.j,  9.20773672e-03+0.j, -9.20773672e-03+0.j,
        9.20773672e-

In [42]:
coeffs = qubit_op.primitive.coeffs

In [43]:
qubit_op.primitive.coeffs[0].real

0.009110140443095271

In [44]:
with open(r'BeH2_hamiltonian.txt', 'w') as fp:
    for i in range(len(paulis)):
        pauli_str = paulis[i].to_label()
        coeffs_str = str(coeffs[i].real)
        fp.write("{}, {}\n".format(pauli_str, coeffs_str))
    print('Done')

Done
